In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr, wilcoxon, kendalltau

%matplotlib inline
pd.set_option('display.precision', 3)

In [ ]:
"""
Load and standardize automated sentiment analysis results.

- Reads sentiment analysis Excel file.
- Renames columns for clarity.
- Adds Sensitivity labels based on QuestionID.
- Converts neutrality scores from [0,1] → [1,5].
- Normalizes selected metrics to [0,1], preserving originals in *_old columns.
"""

import pandas as pd

sentiment_file = "sentiment_analysis_full_results raw.xlsx"
df_auto = pd.read_excel(sentiment_file)

df_auto = df_auto.rename(columns={
    "question_id": "QuestionID",
    "sentiment_consistency_between_arabic_and_english": "AutoConsistency",
    "model": "Model"
})

df_auto["Sensitivity"] = df_auto["QuestionID"].apply(
    lambda x: "Non-sensitive" if x in [0, 1, 2] else "Sensitive"
)

if "NeutralScore_ar" in df_auto.columns:
    df_auto["NeutralScore_ar_std"] = 1 + 4 * df_auto["NeutralScore_ar"]
if "NeutralScore_en" in df_auto.columns:
    df_auto["NeutralScore_en_std"] = 1 + 4 * df_auto["NeutralScore_en"]

for col in ["AutoConsistency", "NeutralScore_ar", "NeutralScore_en"]:
    if col in df_auto.columns:
        df_auto[col + "_old"] = df_auto[col]
        df_auto[col] = (
            df_auto[col] - df_auto[col].min()
        ) / (df_auto[col].max() - df_auto[col].min())

print("Automated analysis data (with standardized metrics):")
display(df_auto)


In [ ]:
"""
Load, reshape, filter, and standardize Qualtrics survey responses.

- Reads a semicolon-delimited CSV of raw survey data.
- Retains metadata columns unchanged.
- Maps question codes to numeric IDs and sensitivity labels.
- Renames response columns to descriptive metric names.
- Concatenates into a long-form DataFrame.
- Converts progress and duration to numeric, filters for completed responses.
- Drops rows missing any key metrics and casts types.
- Standardizes each survey metric to a 0–1 scale, preserving originals.
"""

import pandas as pd

# Path to your survey CSV
survey_file = "Survey Results.csv"

df_survey = pd.read_csv(survey_file, delimiter=";", encoding="utf-8")

meta_cols = [
    "ResponseId", "Status", "IPAddress", "Progress",
    "Finished", "LocationLatitude", "LocationLongitude",
    "Duration (in seconds)"
]

# Map from Qualtrics question codes → (question ID, sensitivity)
question_map = {
    "Q10": (0, "Non-sensitive"),
    "Q8":  (1, "Non-sensitive"),
    "Q9":  (2, "Non-sensitive"),
    "Q1":  (3, "Sensitive"),
    "Q3":  (4, "Sensitive"),
    "Q5":  (5, "Sensitive"),
    "Q6":  (6, "Sensitive"),
    "Q2":  (7, "Sensitive"),
    "Q4":  (8, "Sensitive"),
    "Q7":  (9, "Sensitive"),
}

# Map from Qualtrics suffix → column name
suffix_map = {
    "-1_1": "SentimentConsistency",
    "-1_2": "FactualConsistency",
    "-2_1": "EnglishNeutrality",
    "-2_2": "ArabicNeutrality",
    "-2_3": "QuestionNeutrality",
    "-3":   "Comments",
}

rows = []
for q_code, (qid, sensitivity) in question_map.items():
    q_cols = [c for c in df_survey.columns if c.startswith(q_code + "-")]
    if not q_cols:
        continue
    tmp = df_survey[meta_cols + q_cols].copy()
    rename_dict = {
        c: suffix_map[c.replace(q_code, "")]
        for c in q_cols
        if c.replace(q_code, "") in suffix_map
    }
    tmp = tmp.rename(columns=rename_dict)
    tmp["QuestionID"] = qid
    tmp["Sensitivity"] = sensitivity
    rows.append(tmp)

# Combine into a long DataFrame
long_df = pd.concat(rows, ignore_index=True)

final_cols = meta_cols + [
    "QuestionID", "Sensitivity",
    "FactualConsistency", "SentimentConsistency",
    "EnglishNeutrality", "ArabicNeutrality", "QuestionNeutrality"
]
long_df = long_df[final_cols]

# Convert numeric fields
long_df["Progress"] = pd.to_numeric(long_df["Progress"], errors="coerce")
long_df["Duration (in seconds)"] = pd.to_numeric(long_df["Duration (in seconds)"], errors="coerce")

# Filter out partial/incomplete responses
metrics = [
    "SentimentConsistency", "FactualConsistency",
    "EnglishNeutrality", "ArabicNeutrality", "QuestionNeutrality"
]
valid_df = (
    long_df.loc[
        (long_df["Status"] == "IP Address") &
        (long_df["Progress"] > 60) &
        (long_df["Duration (in seconds)"] > 300)
    ]
    .dropna(subset=metrics)
    .copy()
)

# Cast metrics to integers
for col in metrics:
    valid_df[col] = valid_df[col].astype(int)

# Standardize survey metrics to a 0–1 scale, preserving originals
for col in metrics:
    valid_df[col + "_old"] = valid_df[col]
    valid_df[col] = (
        valid_df[col] - valid_df[col].min()
    ) / (valid_df[col].max() - valid_df[col].min())

print("Survey data (each row is one participant response, with standardized metrics):")
display(valid_df)


In [ ]:
"""
Aggregate automated and survey consistency metrics by question,
generate ranking tables, and compute Spearman's rank correlations.

Steps:
1. Average AutoConsistency by QuestionID → df_auto_agg.
2. Average SurveyConsistency by QuestionID → df_survey_agg.
3. Create overall, non-sensitive, and sensitive ranking tables.
4. Compute Spearman correlations for each grouping.
"""

df_auto_agg = df_auto.groupby("QuestionID", as_index=False).agg({
    "AutoConsistency": "mean",
    "Sensitivity": "first"
})

print("Aggregated Automated Data:")
display(df_auto_agg)

df_survey_agg = valid_df.groupby("QuestionID", as_index=False).agg({
    "SentimentConsistency": "mean",
    "Sensitivity": "first"
}).rename(columns={"SentimentConsistency": "SurveyConsistency"})

print("Aggregated Survey Data:")
display(df_survey_agg)

survey_rank = df_survey_agg.sort_values("SurveyConsistency", ascending=False).reset_index(drop=True)
survey_rank["Rank"] = survey_rank.index + 1
survey_rank = survey_rank.rename(columns={"QuestionID": "SurveyQuestionID"})

auto_rank = df_auto_agg.sort_values("AutoConsistency", ascending=False).reset_index(drop=True)
auto_rank["Rank"] = auto_rank.index + 1
auto_rank = auto_rank.rename(columns={"QuestionID": "AutoQuestionID"})

overall_ranking = pd.merge(
    survey_rank[["Rank", "SurveyQuestionID", "SurveyConsistency", "Sensitivity"]],
    auto_rank[["Rank", "AutoQuestionID", "AutoConsistency"]],
    on="Rank"
)

print("Overall Ranking Table (Rank 1 = most consistent, Rank 10 = least consistent):")
display(overall_ranking)

overall_rho, overall_p = spearmanr(
    df_survey_agg["SurveyConsistency"],
    df_auto_agg["AutoConsistency"]
)
print(f"Overall Spearman's Rank Correlation: rho = {overall_rho:.3f}, p = {overall_p:.4g}")

df_survey_non = df_survey_agg[df_survey_agg["Sensitivity"] == "Non-sensitive"].copy()
df_auto_non   = df_auto_agg[df_auto_agg["Sensitivity"] == "Non-sensitive"].copy()

survey_rank_non = df_survey_non.sort_values("SurveyConsistency", ascending=False).reset_index(drop=True)
survey_rank_non["Rank"] = survey_rank_non.index + 1
survey_rank_non = survey_rank_non.rename(columns={"QuestionID": "SurveyQuestionID"})

auto_rank_non = df_auto_non.sort_values("AutoConsistency", ascending=False).reset_index(drop=True)
auto_rank_non["Rank"] = auto_rank_non.index + 1
auto_rank_non = auto_rank_non.rename(columns={"QuestionID": "AutoQuestionID"})

nonsensitive_ranking = pd.merge(
    survey_rank_non[["Rank", "SurveyQuestionID", "SurveyConsistency", "Sensitivity"]],
    auto_rank_non[["Rank", "AutoQuestionID", "AutoConsistency"]],
    on="Rank"
)

print("\nNon-sensitive Ranking Table:")
display(nonsensitive_ranking)

if len(df_survey_non) > 1 and len(df_auto_non) > 1:
    nonsens_rho, nonsens_p = spearmanr(
        df_survey_non["SurveyConsistency"],
        df_auto_non["AutoConsistency"]
    )
    print(f"Non-sensitive Spearman's Rank Correlation: rho = {nonsens_rho:.3f}, p = {nonsens_p:.4g}")
else:
    print("Not enough non-sensitive data for Spearman correlation.")

df_survey_sens = df_survey_agg[df_survey_agg["Sensitivity"] == "Sensitive"].copy()
df_auto_sens   = df_auto_agg[df_auto_agg["Sensitivity"] == "Sensitive"].copy()

survey_rank_sens = df_survey_sens.sort_values("SurveyConsistency", ascending=False).reset_index(drop=True)
survey_rank_sens["Rank"] = survey_rank_sens.index + 1
survey_rank_sens = survey_rank_sens.rename(columns={"QuestionID": "SurveyQuestionID"})

auto_rank_sens = df_auto_sens.sort_values("AutoConsistency", ascending=False).reset_index(drop=True)
auto_rank_sens["Rank"] = auto_rank_sens.index + 1
auto_rank_sens = auto_rank_sens.rename(columns={"QuestionID": "AutoQuestionID"})

sensitive_ranking = pd.merge(
    survey_rank_sens[["Rank", "SurveyQuestionID", "SurveyConsistency", "Sensitivity"]],
    auto_rank_sens[["Rank", "AutoQuestionID", "AutoConsistency"]],
    on="Rank"
)

print("\nSensitive Ranking Table:")
display(sensitive_ranking)

if len(df_survey_sens) > 1 and len(df_auto_sens) > 1:
    sens_rho, sens_p = spearmanr(
        df_survey_sens["SurveyConsistency"],
        df_auto_sens["AutoConsistency"]
    )
    print(f"Sensitive Spearman's Rank Correlation: rho = {sens_rho:.3f}, p = {sens_p:.4g}")
else:
    print("Not enough sensitive data for Spearman correlation.")


In [ ]:
"""
Compute Pearson correlations between survey and automated standardized metrics.

- Aggregate survey data by QuestionID, averaging standardized survey metrics.
- Aggregate automated data by QuestionID, averaging standardized automated metrics.
- Merge the two aggregated DataFrames on QuestionID.
- Define metric pairs for comparison (survey vs. automated).
- Split into Overall, Non-sensitive, and Sensitive subsets.
- For each subset and metric pair, compute and print Pearson r and p-value.
"""

# Aggregate survey standardized metrics by question (take the mean over responses)
survey_agg = valid_df.groupby("QuestionID", as_index=False).agg({
    "EnglishNeutrality_old": "mean",
    "ArabicNeutrality_old": "mean",
    "SentimentConsistency_old": "mean",
    "Sensitivity": "first"  # assuming sensitivity is consistent per question
})

# Aggregate automated standardized metrics by question (mean across model runs)
auto_agg = df_auto.groupby("QuestionID", as_index=False).agg({
    "NeutralScore_en_std": "mean",
    "NeutralScore_ar_std": "mean",
    "AutoConsistency_old": "mean"
})

# Merge the aggregated data on QuestionID
merged_std = pd.merge(survey_agg, auto_agg, on="QuestionID", how="inner")

# Define the metric pairs for correlation analysis
metrics = {
    "English Neutrality": ("EnglishNeutrality_old", "NeutralScore_en_std"),
    "Arabic Neutrality": ("ArabicNeutrality_old", "NeutralScore_ar_std"),
    "Sentiment Consistency": ("SentimentConsistency_old", "AutoConsistency_old")
}

# Define the data subsets
subsets = {
    "Overall": merged_std,
    "Non-sensitive": merged_std[merged_std["Sensitivity"] == "Non-sensitive"],
    "Sensitive": merged_std[merged_std["Sensitivity"] == "Sensitive"]
}

# Compute and print correlations for each metric pair in each subset
for subset_name, df_subset in subsets.items():
    print(f"=== {subset_name} Data ===")
    # Check if there is enough data to compute correlations
    if len(df_subset) < 2:
        print("Not enough data for correlation analysis in this subset.\n")
        continue
    for metric_name, (survey_col, auto_col) in metrics.items():
        print(f"--- {metric_name} ---")
        # Pearson correlation
        pear_r, pear_p = pearsonr(df_subset[survey_col], df_subset[auto_col])
        print(f"Pearson r: {pear_r:.3f} (p = {pear_p:.4g})")

    print("\n")


In [ ]:
"""
Aggregate and compare survey and automated metrics by question, generate ranking tables,
and compute Spearman's rank correlations for specified metrics.

Defines:
- rbo_ext: compute extrapolated Rank-Biased Overlap between two ranked lists.
- run_ranking_analysis: aggregate data, produce ranking tables, and compute Spearman correlations.

Example analyses at the bottom cover:
1. Sentiment consistency
2. Arabic neutrality
3. English neutrality
"""

def rbo_ext(S, T, p=0.9):
    """
    Compute the extrapolated Rank-Biased Overlap (RBO_ext) between two ranked lists.

    Parameters
    ----------
    S : list
        First ranked list of items.
    T : list
        Second ranked list of items.
    p : float, optional
        Persistence parameter (0 < p < 1) weighting top ranks more heavily.

    Returns
    -------
    float
        Extrapolated RBO value between 0 and 1.
    """
    k = max(len(S), len(T))
    sum_overlap = 0.0
    set_S, set_T = set(), set()
    for d in range(1, k + 1):
        if d <= len(S):
            set_S.add(S[d - 1])
        if d <= len(T):
            set_T.add(T[d - 1])
        overlap = len(set_S & set_T)
        sum_overlap += (p ** (d - 1)) * (overlap / d)
    return (1 - p) * sum_overlap + (p ** k)

def run_ranking_analysis(df_auto, valid_df, survey_col, auto_col, survey_label, auto_label):
    """
    Aggregate survey and automated results by QuestionID, generate ranking tables,
    and compute Spearman's rank correlations for the specified metrics.

    Parameters
    ----------
    df_auto : pandas.DataFrame
        Automated analysis data with 'QuestionID' and 'Sensitivity'.
    valid_df : pandas.DataFrame
        Survey responses with 'QuestionID', 'Sensitivity', and survey_col.
    survey_col : str
        Name of the survey metric column in valid_df.
    auto_col : str
        Name of the automated metric column in df_auto.
    survey_label : str
        Label for the aggregated survey metric in output tables.
    auto_label : str
        Label for the aggregated automated metric in output tables.

    Returns
    -------
    None
        Prints aggregated data, ranking tables, and Spearman correlation results.
    """
    # Aggregate automated data
    df_auto_agg = df_auto.groupby("QuestionID", as_index=False).agg({
        auto_col: "mean",
        "Sensitivity": "first"
    })

    # Aggregate survey data
    df_survey_agg = valid_df.groupby("QuestionID", as_index=False).agg({
        survey_col: "mean",
        "Sensitivity": "first"
    }).rename(columns={survey_col: survey_label})

    print("Aggregated Automated Data:")
    print(df_auto_agg)
    print("\nAggregated Survey Data:")
    print(df_survey_agg)

    # Overall ranking
    survey_rank = df_survey_agg.sort_values(by=survey_label, ascending=False).reset_index(drop=True)
    survey_rank["Rank"] = survey_rank.index + 1
    survey_rank = survey_rank.rename(columns={"QuestionID": "SurveyQuestionID"})
    auto_rank = df_auto_agg.sort_values(by=auto_col, ascending=False).reset_index(drop=True)
    auto_rank["Rank"] = auto_rank.index + 1
    auto_rank = auto_rank.rename(columns={"QuestionID": "AutoQuestionID"})
    overall_ranking = pd.merge(
        survey_rank[["Rank", "SurveyQuestionID", survey_label, "Sensitivity"]],
        auto_rank[["Rank", "AutoQuestionID", auto_col]],
        on="Rank"
    )
    print("\nOverall Ranking Table (Rank 1 = highest):")
    print(overall_ranking)
    overall_rho, overall_p = spearmanr(df_survey_agg[survey_label], df_auto_agg[auto_col])
    print(f"\nOverall Spearman's Rank Correlation: rho = {overall_rho:.3f}, p = {overall_p:.4g}")

    # Non-sensitive ranking
    df_survey_non = df_survey_agg[df_survey_agg["Sensitivity"] == "Non-sensitive"].copy()
    df_auto_non   = df_auto_agg[df_auto_agg["Sensitivity"] == "Non-sensitive"].copy()
    survey_rank_non = df_survey_non.sort_values(by=survey_label, ascending=False).reset_index(drop=True)
    survey_rank_non["Rank"] = survey_rank_non.index + 1
    survey_rank_non = survey_rank_non.rename(columns={"QuestionID": "SurveyQuestionID"})
    auto_rank_non = df_auto_non.sort_values(by=auto_col, ascending=False).reset_index(drop=True)
    auto_rank_non["Rank"] = auto_rank_non.index + 1
    auto_rank_non = auto_rank_non.rename(columns={"QuestionID": "AutoQuestionID"})
    nonsensitive_ranking = pd.merge(
        survey_rank_non[["Rank", "SurveyQuestionID", survey_label, "Sensitivity"]],
        auto_rank_non[["Rank", "AutoQuestionID", auto_col]],
        on="Rank"
    )
    print("\nNon-sensitive Ranking Table:")
    print(nonsensitive_ranking)
    if len(df_survey_non) > 1 and len(df_auto_non) > 1:
        nonsens_rho, nonsens_p = spearmanr(df_survey_non[survey_label], df_auto_non[auto_col])
        print(f"\nNon-sensitive Spearman's Rank Correlation: rho = {nonsens_rho:.3f}, p = {nonsens_p:.4g}")
    else:
        print("\nNot enough non-sensitive data for Spearman correlation.")

    # Sensitive ranking
    df_survey_sens = df_survey_agg[df_survey_agg["Sensitivity"] == "Sensitive"].copy()
    df_auto_sens   = df_auto_agg[df_auto_agg["Sensitivity"] == "Sensitive"].copy()
    survey_rank_sens = df_survey_sens.sort_values(by=survey_label, ascending=False).reset_index(drop=True)
    survey_rank_sens["Rank"] = survey_rank_sens.index + 1
    survey_rank_sens = survey_rank_sens.rename(columns={"QuestionID": "SurveyQuestionID"})
    auto_rank_sens = df_auto_sens.sort_values(by=auto_col, ascending=False).reset_index(drop=True)
    auto_rank_sens["Rank"] = auto_rank_sens.index + 1
    auto_rank_sens = auto_rank_sens.rename(columns={"QuestionID": "AutoQuestionID"})
    sensitive_ranking = pd.merge(
        survey_rank_sens[["Rank", "SurveyQuestionID", survey_label, "Sensitivity"]],
        auto_rank_sens[["Rank", "AutoQuestionID", auto_col]],
        on="Rank"
    )
    print("\nSensitive Ranking Table:")
    print(sensitive_ranking)
    if len(df_survey_sens) > 1 and len(df_auto_sens) > 1:
        sens_rho, sens_p = spearmanr(df_survey_sens[survey_label], df_auto_sens[auto_col])
        print(f"\nSensitive Spearman's Rank Correlation: rho = {sens_rho:.3f}, p = {sens_p:.4g}")
    else:
        print("\nNot enough sensitive data for Spearman correlation.")

if __name__ == "__main__":
    print("=== Analysis for Sentiment Consistency ===")
    run_ranking_analysis(
        df_auto, valid_df,
        survey_col="SentimentConsistency", auto_col="AutoConsistency",
        survey_label="SurveyConsistency", auto_label="AutoConsistency"
    )
    print("\n=== Analysis for Arabic Neutrality ===")
    run_ranking_analysis(
        df_auto, valid_df,
        survey_col="ArabicNeutrality", auto_col="NeutralScore_ar",
        survey_label="SurveyArabicNeutrality", auto_label="AutoNeutral_ar"
    )
    print("\n=== Analysis for English Neutrality ===")
    run_ranking_analysis(
        df_auto, valid_df,
        survey_col="EnglishNeutrality", auto_col="NeutralScore_en",
        survey_label="SurveyEnglishNeutrality", auto_label="AutoNeutral_en"
    )
